
# Linear Discriminant Analysis Using Julia 


In [1]:
# importing libraries 

using LinearAlgebra
using Statistics
using RDatasets
using BenchmarkTools
using CSV
using DataFrames
using Plots

In [2]:
# _class_cov 

function _class_cov(X , y, priors)
```
    Computes within-class covariance matrix
    
    input: 
    X : Predictor matrix 
    y: target values 
    priors: class priors
        
    output:
    cov_ : Weighted within-class covariance matrix
    
```
    classes = sort(unique(y))
    cov_ = zeros(size(X)[2], size(X)[2])
    
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        cov_ += (priors[idx])*(cov(Matrix(Xg)))
    end
    return(cov_)
end

_class_cov (generic function with 1 method)

In [3]:
# mean_col

function mean_col(X)
```
    computes mean of each column 
    
    input:
    X: Input data 
    
    output:
    mean_vec: mean value of each column
    
```
    mean_vec = [mean(c) for c in eachcol(X)]
    return(mean_vec)
end

mean_col (generic function with 1 method)

In [4]:
# _class_Mean

function _class_Mean(X,y)
```
    Computes class mean 
    
    input: 
    X: Input data
    y: target values
    
    output
    
    mean_ :  column wise mean for each class"(a matrix of shape(number of attributes , number of classes))"
    
```
    classes = sort(unique(y))
    
    mean_ = zeros(size(X)[2] ,length(classes) )
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        mean_val = [mean(c) for c in eachcol(Xg)]
        mean_[: ,idx] = mean_val
    end
    return(mean_)
end

_class_Mean (generic function with 1 method)

In [5]:
# _priors 

function _priors(X,y)
```
    Computes class priors 
    
    input: 
    X: Input data
    y: target values
    
    output:
    prior_ :  prior value for each class

```

    classes = sort(unique(y))
    
    prior_ = zeros(length(classes) )
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        prop = size(Xg)[1]/size(X)[1]
        prior_[idx] = prop
    end
    return(prior_)
end

_priors (generic function with 1 method)

In [6]:
function _make_cat(y)
```
    transform target variable into numerical categorical varible 
    
    input:
    y: target variable
    
    output:
    my_arr: numerical values corresponding to each categorical varible"(1:number of classes)"
    
```
    classes = sort(unique(y))
    
    labels = [x for x in range(1,length(classes))]
    
    my_arr = [0 for x in range(1, length(y))]
    
    for (id ,val) in enumerate(y)
        for (idx, class) in enumerate(classes)
            if val == class
                my_arr[id] = idx
            end
        end
    end
    return(my_arr)
end

_make_cat (generic function with 1 method)

In [7]:
# _solve_lsqr
```
    Computes a straightforward solution of the optimal decision rule based directly on the discriminant functions.
       using formula
    coef_ = inv"(sigma)""*"_class_Mean
    
    Note: dimensionality reduction is not supported with this solver
    
    input: 
    X : Predictor matrix 
    y: target values 
    priors: class priors
        
    output:
    coef_ : coefficient of discriminant functions
    intercept_ :  intercept of discriminant functions
    
    
```


function _solve_lsqr(X,y , priors::Union{Vector, Nothing}=nothing)
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]
    
    coef_ = inv(cov_)*mean_
    intercept_ =  -0.5 * diag(transpose(mean_)*coef_) + ln_priors
    
    return(Dict("coef" => coef_, "intercept" => intercept_))
end

_solve_lsqr (generic function with 2 methods)

In [8]:

function _solve_eigen(X,y ,priors::Union{Vector, Nothing}=nothing )

```
    Computes the optimal solution of the Rayleigh coefficient"(J(W))", "(basically the ratio of 
    between class scatter to within class scatter)" 
    "J(W) = \frac{W.T*Sb*W}{W.T*Sw*W}"
    
    This solver supports both classification and dimensionality reduction
    
    input: 
    X : Predictor matrix 
    y: target values 
    priors: class priors
        
    output:
    coef_ : coefficient of discriminant functions
    intercept_ :  intercept of discriminant functions
       
```
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]
    
    Sw = cov_  # within scatter
    St = cov(Matrix(X))  # total scatter
    Sb = St - Sw
    
    evals , evecs = eigen(Sb, Sw)
    
    coef_ = (transpose(mean_)*evecs)*transpose(evecs)
    intercept_ =  -0.5 * diag(transpose(mean_)*transpose(coef_)) + ln_priors
    
    return(Dict("coef" => coef_, "intercept" => intercept_))
end

_solve_eigen (generic function with 2 methods)

In [9]:
function _solve_SVD(X,y , tol ,priors::Union{Vector, Nothing}=nothing)
```
    SVD solver
    
    This solver supports both classification and dimensionality reduction
    
    input: 
    X : Predictor matrix 
    y: target values 
    tol: tolerance value for cut-off eigen value of diag matrix of SVD 
    priors: class priors
    
    output:
    pramas:
        intercept: intercept values of discriminant functions 
        coef: coef of discriminant functions
        rank: rank 
        scalings: transformation matrix"(W)"
    
    
```
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    n_samples = size(X)[1]
    classes = sort(unique(y))
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]


    Xc = []
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        Xg_mean = broadcast(- , Xg , transpose(mean_[:,idx]))
        push!(Xc , Matrix(Xg_mean))
    end 

    xbar_ = mean_col(X)

    Xc = vcat(Xc...)

    std_ =  std(Xc , dims=1)
    std_[std_.==0] .= 1

    fac = 1.0 / (size(X)[1] - length(classes))

    X_temp = sqrt(fac)*broadcast(/, Xc , std_)

    U, S, Vt = svd(X_temp)

    S_diag = S[S.>=tol]
    rank = length(S_diag)
    scalings_temp = broadcast(/, Vt[:, 1:rank] , transpose(std_))
    scalings = broadcast(/, scalings_temp , transpose(S[1:rank]))

    n_classes = length(classes)
    fac_2 = ifelse.( n_classes==1 , 1, 1/(n_classes-1))
    temp_1 = [sqrt(val) for val in (n_samples * priors *fac_2) ]
    temp_2 = broadcast(-, mean_ , xbar_)
    temp_3 = broadcast(* , transpose(temp_2) , temp_1)

    X_new = temp_3*scalings

    U, S, Vt = svd(X_new, full=false)

    rank = length(S[S.>=tol])
    scalings_ = scalings*Vt[:, 1:rank]

    coef = transpose(temp_2)*scalings_

    intercept_ = (-0.5)*sum(coef.*coef , dims=2) + ln_priors
    coef_ = coef*transpose(scalings_)
    intercept_ = intercept_ - coef_*xbar_
    
    
    if length(classes) ==2
        coef_ = coef_[2,:] - coef_[1,:]
        intercept_ = intercept_[2] -  intercept_[1]
    end
    
    params = Dict("intercept" => intercept_ , "coef" => coef_, "rank" => rank, "scalings" => scalings_)

    return(params)
end





_solve_SVD (generic function with 2 methods)

In [10]:


function _solve_moment(X,y , tol ,priors::Union{Vector, Nothing}=nothing)

```
    SVD solver, uses standard estimators of the mean and variance "(replicating R's solution)"
    
    This solver supports both classification and dimensionality reduction
    
    input: 
    X : Predictor matrix 
    y: target values 
    tol: tolerance value for cut-off eigen value of diag matrix of SVD 
    priors: class priors
    
    output:
    pramas:
        intercept: intercept values of discriminant functions 
        coef: coef of discriminant functions
        rank: rank 
        scalings:  transformation matrix"(W)"
```
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    n_samples = size(X)[1]
    classes = sort(unique(y))
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]

    Xc = []
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        Xg_mean = broadcast(- , Xg , transpose(mean_[:,idx]))
        push!(Xc , Matrix(Xg_mean))
    end

    xbar_ = mean_col(X)

    Xc = vcat(Xc...)

    std_ =  std(Xc , dims=1)
    std_[std_.==0] .= 1

    fac = 1.0 / (size(X)[1] - length(classes))
    X_temp = sqrt(fac)*broadcast(/, Xc , std_)
    
    
    U, S, Vt = svd(X_temp)
    S_diag = S[S.>=tol]
    rank = length(S_diag)
    scalings_temp = broadcast(/, Vt[:, 1:rank] , transpose(std_))
    scalings = broadcast(/, scalings_temp , transpose(S[1:rank]))

    n_classes = length(classes)
    fac_2 = ifelse.( n_classes==1 , 1, 1/(n_classes-1))

    temp_1 = [sqrt(val) for val in (n_samples * priors *fac_2) ]
    temp_2 = broadcast(-, mean_ , xbar_)
    temp_3 = broadcast(* , transpose(temp_2) , temp_1)

    X_new = temp_3*scalings

    U, S, Vt = svd(X_new, full=false)
    rank  = length(S[S.>=tol])
    scalings_ = scalings*Vt[:, 1:rank]
    intercept_temp_1 = transpose(temp_2)*scalings_
    intercept_ = (-0.5)*sum(intercept_temp_1.*intercept_temp_1 , dims=2) + ln_priors
    intercept_temp_2 = intercept_temp_1*transpose(scalings_)
    intercept_ = intercept_ - intercept_temp_2*xbar_
    params = Dict("intercept" => intercept_ , "coef" => scalings_, "rank" => rank, "scalings" => scalings_)

    return(params)
end


_solve_moment (generic function with 2 methods)

In [11]:
function fit_(X, y , tol, method , priors::Union{Vector, Nothing}=nothing)

```
    applies LDA algo based on method given  
    input: 
    X : Predictor matrix 
    y: target values 
    tol: tolerance value for cut-off eigen value of diag matrix of SVD 
    priors: class priors
    method: method for evaluating coeff and intercept
    
    output:
    pramas:
        intercept: intercept values of discriminant functions 
        coef: coef of discriminant functions
        rank: rank 
        scalings:  transformation matrix"(W)"
```
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    if method == "SVD"
        return(_solve_SVD(X, y, tol ,priors))  
        
    elseif method == "lsqr"
        return( _solve_lsqr(X,y , priors))
    
    elseif method == "eigen" || "moment"
        return(_solve_eigen(X,y ,priors))
    
    end
end
    
        
        

fit_ (generic function with 2 methods)

In [12]:
function decision_(X , y ,tol, method , priors::Union{Vector, Nothing}=nothing)
    
```
    Evaluate W^T*X + b  
    
    input: 
    X : Predictor matrix 
    y: target values 
    tol: tolerance value for cut-off eigen value of diag matrix of SVD 
    method: method for evaluating coeff and intercept
    priors: class priors
    
    output:
    score: W^T*X + b  matrix
    
```
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    
        
    model = fit_(X, y , tol, method , priors)
    
    coef_ = model["coef"]
    intercept_ = model["intercept"]
    
    if length(priors) == 2
        scores = broadcast(+, Matrix(X)*transpose(transpose(coef_)) , intercept_)
    
    else    
        num_lines = length(coef_)
        scores = broadcast(+ , transpose(Matrix(X)*transpose(coef_)) , intercept_)
    end
    return(scores)
end   

decision_ (generic function with 2 methods)

In [13]:
function minus_max(X)
    
    max_val = findmax(X, dims = 1)[1]
    
    for i in range(1, size(X)[2])
        for j in range(1, size(X)[1])
            X[j,i] = X[j,i] - max_val[i]
        end
    end
    return(X)
end
    

minus_max (generic function with 1 method)

In [14]:
function softmax_(score)
    
    score = minus_max(score)
    
    map_exp = map(exp , score)
    sum_val = [sum(c) for c in eachcol(map_exp)]
    
    scores =  broadcast(/ , map_exp , transpose(sum_val) )
    
    return(scores)
    
end

softmax_ (generic function with 1 method)

In [63]:
function predictProba(X,y, model, tol ,priors::Union{Vector, Nothing}=nothing)
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    if length(priors) ==2
        
        scores = decision_(X , y ,tol, method , priors)

        return([(1/(1+exp(-x)), 1- (1/(1+exp(-x)))) for x in scores])
        
    else
        score = decision_(X , y ,tol, method , priors)
        
        return(softmax_(score))
    end
end
    

predictProba (generic function with 2 methods)

In [32]:
function predictClass(X,y, model, tol ,priors::Union{Vector, Nothing}=nothing)
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    scores = predictProba(X,y, model, tol ,priors)
    
    max_val = findmax(scores , dims =1 )
    
    classes = []
    
    if length(priors) == 2
        
        for i in range(1, size(X)[1])
            if scores[i][1] > scores[i][2]
                append!(classes, 1)
            else
                append!(classes, 2)
            end
        end
        
    elseif length(priors) > 2
        for i in range(1, size(X)[1])
            class = max_val[2][i][1]
            append!(classes, class)
        end
    end 
    
    if typeof(y[1]) == Int64
        return(classes)
    else
        class_list = sort(unique(y))
        classes_cat = [ class_list[i] for i in classes]
        return(classes_cat)
    end
end
    

predictClass (generic function with 2 methods)

# Test

### a) IRIS

In [16]:
iris = dataset("datasets", "iris")

X = iris[:,1:4]

labels = [0 for y in 1:150]
for i = 1:size(iris, 1)
    x = iris[:, 5][i]
    if x == "setosa"
        labels[i] = 1
    elseif x == "virginica"
        labels[i] = 2
    else
        labels[i] = 3
    end
end

y = labels;

In [142]:
params_ = _solve_eigen(X,y)
params_["intercept"]

3-element Vector{Float64}:
  -86.30846997367402
 -104.36831998644988
  -72.85260740064221

In [143]:
tol = 0.0001
paramas_iris = _solve_SVD(X,y , tol)
paramas_iris["coef"]

3×4 Matrix{Float64}:
  6.31476  12.1393   -16.9464   -20.7701
 -4.78356  -7.76327   12.2508    17.7075
 -1.5312   -4.37604    4.69567    3.06259

In [14]:
tol = 0.0001
@benchmark pramas = _solve_SVD(X,y , tol)

BenchmarkTools.Trial: 8679 samples with 1 evaluation.
 Range (min … max):  344.229 μs …  10.528 ms  ┊ GC (min … max): 0.00% … 85.00%
 Time  (median):     449.235 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   565.786 μs ± 504.434 μs  ┊ GC (mean ± σ):  4.72% ±  5.60%

  ██▂                                                            
  ███▇▅▄▃▃▃▃▃▂▂▂▄▃▃▅▆▅▄▃▃▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  344 μs           Histogram: frequency by time         1.44 ms <

 Memory estimate: 208.69 KiB, allocs estimate: 1178.

In [59]:
pramas = _solve_SVD(X,y , tol)
proj = pramas["coef"]

make_normal(transpose(proj))

4×3 Matrix{Float64}:
  0.209815  -0.20457   -0.210479
  0.403343  -0.331998  -0.601533
 -0.563065   0.523907   0.645468
 -0.690109   0.757265   0.420984

# Python's result on IRIS LDA : 
### Coef_

[[  6.31475846,  12.13931718, -16.94642465, -20.77005459],
[ -1.53119919,  -4.37604348,   4.69566531,   3.06258539],
[ -4.78355927,  -7.7632737 ,  12.25075935,  17.7074692 ]]

In [61]:
pramas["intercept"]

3×1 Matrix{Float64}:
 -15.477836726795022
 -33.53768673957079
  -2.0219741537631464

### Intercept_
[-15.47783673,  -2.02197415, -33.53768674]

In [65]:
pramas["scalings"]

4×2 Matrix{Float64}:
  0.829378   0.0241021
  1.53447    2.16452
 -2.20121   -0.931921
 -2.81046    2.83919

In [17]:
@benchmark p = _solve_moment(X,y , 0.0001)

BenchmarkTools.Trial: 8041 samples with 1 evaluation.
 Range (min … max):  352.914 μs …  32.721 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     562.925 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   610.044 μs ± 714.985 μs  ┊ GC (mean ± σ):  4.46% ± 5.73%

  █▅                                                             
  ██▆▄▃▃▃▃▃▃▃▃▃▆▄▄██▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  353 μs           Histogram: frequency by time         1.56 ms <

 Memory estimate: 229.44 KiB, allocs estimate: 1223.

In [18]:
params_R_iris = _solve_moment(X,y ,
    0.0001)
params_R_iris["coef"]

4×2 Matrix{Float64}:
  0.829378   0.0241021
  1.53447    2.16452
 -2.20121   -0.931921
 -2.81046    2.83919

In [20]:
function make_normal(X)
    
    temp = X.*X
    temp = sum(temp , dims = 1)
    temp = .√(temp)
    temp = broadcast(/ , X , temp)
    return(temp)
end

make_normal (generic function with 1 method)

In [21]:
make_normal(proj)

4×2 Matrix{Float64}:
  0.208742   0.00653196
  0.386204   0.586611
 -0.554012  -0.252562
 -0.70735    0.769453

###  R's result on IRIS LDA

Coefficients of linear discriminants:
                    LD1         LD2
Sepal.Length  0.8293776  0.02410215
Sepal.Width   1.5344731  2.16452123
Petal.Length -2.2012117 -0.93192121
Petal.Width  -2.8104603  2.83918785


## Penguin

In [22]:
using CSV
using DataFrames

df = CSV.read("penguins_lter.csv" , DataFrame)
df = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)","Sex" ]]
df = dropmissing(df::AbstractDataFrame)
y = df[!,"Sex"]
labels = zeros(length(y))
for (idx,sex) in enumerate(y)
    if sex =="MALE"
        labels[idx] = 1
    end
end
y = labels
X = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]];


In [23]:
df = CSV.read("file1.csv" , DataFrame)

X = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]]

y = df[!,"Species"]
labels = zeros(length(y))
for (idx,specie) in enumerate(y)
    if specie =="Adelie Penguin (Pygoscelis adeliae)"
        labels[idx] = 1
    elseif specie == "Gentoo penguin (Pygoscelis papua)"
        labels[idx] = 2
    end
end
y = labels;


In [24]:
params_R_penguin = _solve_moment(X,y,
    0.0001 )
params_R_penguin["coef"]

4×2 Matrix{Float64}:
  0.0883267   -0.417871
 -1.0373      -0.0210049
  0.0861628    0.0134747
  0.00129952   0.00171144

In [25]:
params_R_penguin["coef"]

4×2 Matrix{Float64}:
  0.0883267   -0.417871
 -1.0373      -0.0210049
  0.0861628    0.0134747
  0.00129952   0.00171144


4×2 Matrix{Float64}:
  0.0820607   -0.998165
 -0.991605    -0.052959
  0.0999184    0.0290805
  0.00133546   0.00413342


4×2 Matrix{Float64}:
  0.084554    -0.998213
 -0.992998    -0.0501766
  0.0824825    0.0321884
  0.00124401   0.00408829


Coefficients of linear discriminants:
                            LD1          LD2
Culmen.Length..mm.   0.08832666 -0.417870885
Culmen.Depth..mm.   -1.03730494 -0.021004854
Flipper.Length..mm.  0.08616282  0.013474680
Body.Mass..g.        0.00129952  0.001711436


Coefficients of linear discriminants:
                            LD1          LD2
Culmen.Length..mm.   0.08832666 -0.417870885
Culmen.Depth..mm.   -1.03730494 -0.021004854
Flipper.Length..mm.  0.08616282  0.013474680
Body.Mass..g.        0.00129952  0.001711436



In [28]:
params_R = _solve_moment(X,y , 0.0001 )

Dict{String, Any} with 4 entries:
  "rank"      => 2
  "intercept" => [-16.7292; 32.1089; -56.9873;;]
  "coef"      => [0.0883267 -0.417871; -1.0373 -0.0210049; 0.0861628 0.0134747;…
  "scalings"  => [0.0883267 -0.417871; -1.0373 -0.0210049; 0.0861628 0.0134747;…

##  R's result on PENGUIN LDA

Coefficients of linear discriminants:
                           LD1
bill_length_mm     0.036231765
bill_depth_mm      0.751144512
flipper_length_mm -0.005785753
body_mass_g        0.001889799

In [29]:
tol = 0.0001
pramas_py_penguin = _solve_SVD(X,y , tol)
pramas_py_penguin["coef"]

3×4 Matrix{Float64}:
  1.06215    2.06466  -0.206102  -0.00755752
 -0.755773   3.38789  -0.268351  -0.0023679
  0.340615  -5.30055   0.443381   0.00708508

## Python's ouptut
array([[ 1.06214785e+00,  2.06466324e+00, -2.06101568e-01,
        -7.55752064e-03],
       [-7.55773193e-01,  3.38788801e+00, -2.68350550e-01,
        -2.36789997e-03],
       [ 3.40615434e-01, -5.30055439e+00,  4.43380810e-01,
         7.08507560e-03]])

In [115]:
df = CSV.read("file2.csv" , DataFrame)

X = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]]

y = df[!,"Sex"]

labels = zeros(length(y))
for (idx,sex) in enumerate(y)
    if sex =="MALE"
        labels[idx] = 1
    end
end
y = labels;


In [33]:
#params = _solve_SVD(X,y , 0.0001 )

@benchmark pramas = _solve_SVD(X,y , tol)

BenchmarkTools.Trial: 8181 samples with 1 evaluation.
 Range (min … max):  374.626 μs …  30.925 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     451.998 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   601.898 μs ± 992.726 μs  ┊ GC (mean ± σ):  7.28% ± 6.89%

  █▇▆▆▆▅▅▄▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▁▂▁▁▁▁▁                               ▂
  ██████████████████████████████████▇▇█▇▇▆▆▇▆▆▅▅▅▆▅▅▅▄▃▅▁▅▅▃▁▁▄ █
  375 μs        Histogram: log(frequency) by time       1.58 ms <

 Memory estimate: 316.94 KiB, allocs estimate: 997.

In [118]:
params_ = _solve_SVD(X,y , tol)
params_["coef"]

4-element Vector{Float64}:
  0.09300704575251453
  1.8569412293410679
 -0.0172074752474619
  0.004687819990745008

In [123]:
params_["scalings"]

4×1 Matrix{Float64}:
  0.037558817726336936
  0.749884232931413
 -0.006948854477859845
  0.0018930713812239334

## Python's  result on Penguins LDA

### Coef_

array([[ 0.09300705,  1.85694123, -0.01720748,  0.00468782]])

In [120]:
params_["intercept"]

-52.199787156462286

### intercept_

array([52.19978716])

In [124]:
params_R = _solve_moment(X,y , tol)

params_R["coef"]
params_R["intercept"]

4×1 Matrix{Float64}:
  0.037558817726336936
  0.749884232931413
 -0.006948854477859845
  0.0018930713812239334

## MPG

In [128]:
mpg = dataset("ggplot2", "mpg")

X = mpg[!,["Displ", "Cyl"]]
y = _make_cat(mpg[!,"Drv"]);


In [38]:
@benchmark pramas_mpg = _solve_moment(X,y , 0.0000001 )

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  316.201 μs …  14.424 ms  ┊ GC (min … max): 0.00% … 89.60%
 Time  (median):     334.360 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   412.194 μs ± 434.863 μs  ┊ GC (mean ± σ):  5.57% ±  5.57%

  ██▅▄▃▃▃▃▂▁▁▁▁▁▁▂▁▂▂▂▁▁▁▁                                      ▂
  ██████████████████████████▇▇▇▇▇▆▇▆▆▅▆▅▅▆▅▅▅▅▆▆▅▆▄▆▆▄▄▅▅▄▄▄▁▄▅ █
  316 μs        Histogram: log(frequency) by time       1.17 ms <

 Memory estimate: 208.50 KiB, allocs estimate: 1083.

In [55]:
pramas_mpg = _solve_moment(X,y , 0.0000001 )
pramas_mpg["intercept"]

3×1 Matrix{Float64}:
 -2.85861697736825
  1.798512608315058
 -9.46405045559477

### R's result 
Coefficients of linear discriminants:
             LD1       LD2
displ -1.5267671  1.803505
cyl    0.3884603 -1.662712

# Test of predict function

## Iris 

In [33]:
## Iris 

iris = dataset("datasets", "iris")

X_iris = iris[:,1:4]

labels = [0 for y in 1:150]
for i = 1:size(iris, 1)
    x = iris[:, 5][i]
    if x == "setosa"
        labels[i] = 1
    elseif x == "virginica"
        labels[i] = 2
    else
        labels[i] = 3
    end
end

y_iris = labels;

In [34]:
# Decision function:

# it returns W^T*X +b 

decision_(X_iris,y_iris , 0.0001 , "SVD")

3×150 Matrix{Float64}:
  31.336    24.0034   26.863    21.6283  …  -67.6755   -74.3345   -65.6157
 -64.4127  -59.5744  -61.3954  -57.6906      11.1982    17.2904     9.30181
 -17.9608  -15.4665  -16.5051  -14.9752       5.43973    6.00658    5.27637

In [39]:
## Predict probability:

## It predicts probably of each instance of being in each class using softmax function 
predictProba(X_iris ,y_iris, "SVD" ,0.0001)

3×150 Matrix{Float64}:
 1.0          1.0          1.0          …  1.61369e-40  2.85801e-33
 2.61117e-42  5.04214e-37  4.67593e-39     0.999987     0.982458
 3.89636e-22  7.21797e-18  1.46385e-19     1.25747e-5   0.0175423

In [41]:
# Predict class 
# Its predicts the class based on probability values. 

predictClass(X_iris,y_iris, "SVD", 0.0001);

### Here I am saving the result in CSV for better analysis :

In [46]:
using CSV
res_iris = predictClass(X_iris,y_iris, "SVD", 0.0001);
res_iris_cat = predictClass(X_iris,iris[:,5], "SVD", 0.0001);

iris[: ,"result_iris_cat"] = res_iris_cat

CSV.write("result_iris_cat.csv", iris)

"result_iris_cat.csv"

## MPG

In [47]:
mpg = dataset("ggplot2", "mpg")

X_mpg = mpg[!,["Displ", "Cyl"]]
y_mpg = _make_cat(mpg[!,"Drv"]);

priors = _priors(X_mpg,y_mpg)
tol = 0.0001
method = "SVD"
model = fit_(X_mpg, y_mpg , tol, method , priors)

Dict{String, Any} with 4 entries:
  "rank"      => 2
  "intercept" => [-2.85862; 1.79851; -9.46405;;]
  "coef"      => [0.79579 -0.150989; -1.50024 0.35878; 3.08237 -0.899155]
  "scalings"  => [-1.52677 1.80351; 0.38846 -1.66271]

In [48]:
predictProba(X_mpg,mpg[!,"Drv"], "SVD", 0.0001)

3×234 Matrix{Float64}:
 0.0715136    0.0715136    0.108617    …  0.215958    0.215958    0.611966
 0.928189     0.928189     0.890669       0.782062    0.782062    0.353083
 0.000297495  0.000297495  0.00071384     0.00197998  0.00197998  0.0349511

In [51]:
predict = predictClass(X_mpg,y_mpg, "SVD", 0.0001)

234-element Vector{Any}:
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 ⋮
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 1

In [50]:
cat_predict = predictClass(X_mpg,mpg[!,"Drv"], "SVD", 0.0001)

234-element Vector{String}:
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 ⋮
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "f"
 "4"

## Penguin 

In [52]:
df = CSV.read("file1.csv" , DataFrame)

X_penguin = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]]

y_penguin = df[!,"Species"]

labels = zeros(length(y))
for (idx,specie) in enumerate(y)
    if specie =="Adelie Penguin (Pygoscelis adeliae)"
        labels[idx] = 1
    elseif specie == "Gentoo penguin (Pygoscelis papua)"
        labels[idx] = 2
    end
end
y = labels;

In [53]:
decision_(X_penguin,y_penguin , 0.0001 , "SVD")

3×342 Matrix{Float64}:
   8.4606     2.29389    2.60919    9.79735  …  -25.9815  -21.1147  -21.0045
  -2.23511   -5.9027    -1.51246   -3.75143     -19.992   -21.1558  -15.1972
 -35.9687   -26.3705   -29.1847   -36.7714       16.1307   10.7994    7.37

In [54]:
predictProba(X_penguin,y_penguin, "SVD", 0.0001)

3×342 Matrix{Float64}:
 0.999977     0.999724     0.984041     …  1.37997e-14  4.75453e-13
 2.26413e-5   0.000275516  0.0159588       1.32447e-14  1.58194e-10
 5.06537e-20  3.55699e-13  1.53139e-14     1.0          1.0

In [55]:
predictClass(X_penguin,y_penguin, "SVD", 0.0001)

342-element Vector{String}:
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 "Adelie Penguin (Pygoscelis adeliae)"
 ⋮
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"
 "Gentoo penguin (Pygoscelis papua)"

# Penguin (2 classes)

In [60]:
df = CSV.read("file2.csv" , DataFrame)

X_penguin_2 = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]]

y = df[!,"Sex"]

labels =  [0 for x in range(1, length(y))]
for (idx,sex) in enumerate(y)
    if sex =="MALE"
        labels[idx] = 1
    else
        labels[idx] = 0
    end
end
y = labels;

In [61]:
decision_(X_penguin_2, y , 0.0001 , "SVD")

334-element Vector{Float64}:
  0.6263612666421778
 -1.6021058899002654
 -3.146703787830525
 -0.09552660575226213
  3.5495017352390548
 -1.6494647477584152
  6.402237915599059
 -3.8257684642851757
  5.284527064181219
  7.419044425951832
 -1.5806243546731622
 -0.5010098335444795
  7.897013069982229
  ⋮
  5.909346381799942
 -1.7953032509887734
  4.683508380232929
 -4.944799978176881
  6.101988372900415
  0.21210400636205406
  6.728265031119221
 -2.964646003508655
 -2.256478058758354
  4.776650691966459
  0.2155407052112963
  3.9870539412930412

In [64]:
prob = predictProba(X_penguin_2,y, model, tol);

In [66]:
predictClass(X_penguin_2,y, model, tol)

334-element Vector{Any}:
 1
 2
 2
 2
 1
 2
 1
 2
 1
 1
 2
 2
 1
 ⋮
 1
 2
 1
 2
 1
 1
 1
 2
 2
 1
 1
 1